In [1]:
def eR(R,B,W):
    for j in range(L,1,-1):
        R[j-1]=np.einsum("dja,lbjj,mjc,abc->dlm",B[j],W,np.conj(B[j]),R[j])
    return R    
def Right_canonical(Fs,L,d=2):
  B,vi,k,v=[],[[1.]],ka(L),[]
  B.append([1.])   #LAMda
  v.append([1.])
  for j in range(L):
    Fs=Fs.reshape(d*k[j],-1)
    S,V,D=truncate(j,Fs,k)
    v.append(V)
    vi.append(np.divide(1.0, V, out=np.zeros_like(V), where=np.abs(V)>=1E-14))
    B.append(np.einsum("a,aib,b->aib",vi[j],S,V))
    Fs=np.einsum('a,aib->aib',V,D)
  return B,v
def ka(L,exact=False,cap = 64):
  if exact:
     k=np.zeros(L+1,dtype=int)
     for j in range(L+1):
      if j<L/2:
       k[j]=int(pow(2,j))
      else:
       k[j]=int(pow(2,L-j))
     return k
  else:
     k=np.zeros(L+1,dtype=int)
     for j in range(L+1):
      if j<L/2:
       k[j]=min(int(pow(2,j)),cap)
      else:
       k[j]=min(int(pow(2,L-j)),cap)
     return k
def Left_canonical(Fs,L,d=2):
  A,k=[],ka(L)
  for j in range(L):
    Fs=Fs.reshape(k[j]*d,-1)
    S,V,D=truncate(j,Fs,k,d=2)
    Fs=np.einsum("a,aib->aib",V,D)
    A.append(S)
  A.append(np.array([1.]))
  return A,np.array([1.])

def MPS(Fs,L,n,chi,d=2):
 if n==0:
    AB,v=Right_canonical(Fs,L)
 elif n==L:
   AB,v=Left_canonical(Fs,L)
 else:
  AB,v,k=[],[[1.]],ka(L,cap = chi)
  for j in range(L):
    Fs=Fs.reshape(2*k[j],-1)
    #S,V,D=LA.svd(Fs,full_matrices=False)
    if j<n:
      S,V,D=truncate(j,Fs,k)

      #AB.append(S.reshape(k[j],2,k[j+1]))
      AB.append(S)
      v.append(V)
      Fs=np.einsum("j,jkd->jkd",V,D)
    else:
      S,V,D=truncate(j,Fs,k)

      vi=np.divide(1.0, v[j], out=np.zeros_like(v[j]), where=np.abs(v[j])>=1E-14)
      if j==n:
       AB.append(v[j])
      AB.append(np.einsum("a,aib,b->aib",vi,S,V))
      Fs=np.einsum("a,aib->aib",V,D)
 return AB,v 

def truncate(j,Theta,k,d=2):
    S,V,D=LA.svd(Theta,full_matrices=False)
    
    lam = V[:k[j+1]]/LA.norm(V[:k[j+1]])
   
    A=S[:,:k[j+1]].reshape(k[j],d,k[j+1])
    if j!=L-1:
     B=D[:k[j+1]].reshape(k[j+1],d,-1)
    else:
      B=np.array([[[1.]]])
    return A,lam,B
# B representation of the MPS
def B_to_Fs(B,L,d=2):
    Fs=np.diag(B[0])   
    for j in range(1,L+1,):
        Fs=np.einsum("...i,ijk->...jk",Fs,B[j])
    Fs=Fs.reshape(pow(2,L))
    return Fs
# A representation of the MPS
def A_to_Fs(A,L,d=2):
  Fs=np.diag(A[len(A)-1])  
 
  for j in range(L-1,-1,-1):
    Fs=np.einsum("ijk,k...->ij...",A[j],Fs) 

  Fs=Fs.reshape(pow(2,L))
  return Fs
def Scalar_multip(Fs,L,O,d=2):
    s=0.
    for j in range(len(O)):
       if len(O[j])==2:
        n=O[j][0]
        AB=MPS(Fs,L,n)
        s+=np.einsum("i,ijk,jm,imk,i->",AB[n],AB[n+1],O[j][1],np.conj(AB[n+1]),AB[n]).real
       elif len(O[j])==3:
        n=O[j][0]
        AB=MPS(Fs,L,n)
        s+=np.einsum("i,ijk,kyu,jm,yb,imo,obu,i->",AB[n],AB[n+1],AB[n+2],O[j][1],O[j][2],np.conj(AB[n+1]),np.conj(AB[n+2]),AB[n]).real
    return s/L
def H_mult(v):
    return np.matmul(H,v)

In [2]:
import numpy as np
from numpy import linalg as LA
from quspin.basis import spin_basis_1d 
from quspin.operators import hamiltonian # Hamiltonians and operators
from numpy.random import random as rnd
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import LinearOperator

np.random.seed(0)
data_type=np.complex128

chi = 64          # Maximal bond dimension                            
Jzz=0.1            # ZZ interaction  
chi_MPO=3        # MPO dimension
hx=0.97           # X field

L=10               # system size
basis = spin_basis_1d(L,pauli=True) # Hilbert space-boson basis
k=ka(L,cap = chi)
d=2


Fs=np.array([rnd()+rnd()*1j for j in range(2**L)])
Fs=(Fs/LA.norm(Fs)).astype(complex)

n=10   # Number of zips


S_z = np.diag([1.,-1.]).astype(data_type)
S_x =np.array([[0.,1.],[1.,0.]],dtype=data_type)
S_y=np.array([[0.,complex(0,-1.)],[complex(0,1.),0.]],dtype=data_type) #Pauli Matrices

W = np.zeros((chi_MPO,chi_MPO,d,d),dtype=data_type)

W[0,0] = np.array(np.eye(2),dtype=data_type)
W[0,1] = S_z
W[0,2] = hx*S_x
W[1,2] = Jzz*S_z
W[2,2] = np.array(np.eye(2),dtype=data_type) # MPO rep

# Left and Right vectors
v_L,v_R=np.zeros(chi_MPO,dtype=np.complex_),np.zeros(chi_MPO,dtype=np.complex_)
v_L[0],v_R[-1]=1.+0j,1.+0j

# Initialise Left and Right environments
L_env=[np.zeros((k[j],chi_MPO,k[j]),dtype=np.complex_)for j in range(L+1)]
R_env=[np.zeros((k[j],chi_MPO,k[j]),dtype=np.complex_)for j in range(L+1)]

L_env[0]=np.einsum("ab,c->acb",np.eye(k[0],dtype=np.complex_),v_L)
R_env[-1]=np.einsum("ab,c->acb",np.eye(k[-1],dtype=np.complex_),v_R)

# Create right and left MPS
Be,lamb=MPS(Fs,L,0,chi)
Ae,lama=MPS(Fs,L,L,chi)


for j in range(L,1,-1):
    R_env[j-1]=np.einsum("dja,ebji->debia",Be[j],W)
    R_env[j-1]=np.einsum("debia,fic->debfca",R_env[j-1],np.conj(Be[j]))
    R_env[j-1]=np.einsum("debfca,abc->def",R_env[j-1],R_env[j])

vi=[]
for j in range(len(lamb)):
  vi.append(np.divide(1.0, lamb[j], out=np.zeros_like(lamb[j]), where=np.abs(lamb[j])>=1E-14))

#DMRG cycle
for i in range(n):
  #Right sweep
  for j in range(L-2):
    # Evaluate the Hamiltonian
    H=np.einsum("abc,bdij->adijc",L_env[j],W)
    H=np.einsum("adijc,dekl->aekl ijc",H,W)
    H=np.einsum("aekl ijc,feg->aikf cjlg",H,R_env[j+2]).reshape((k[j]*k[j+2]*d**2,k[j]*k[j+2]*d**2))

    A = LinearOperator((k[j]*k[j+2]*d**2,k[j]*k[j+2]*d**2), matvec=H_mult)
    Theta=np.einsum("a,aib,bjc->aijc",lamb[j],Be[j+1],Be[j+2]).reshape(k[j]*k[j+2]*d**2)
    
    w, v = eigsh(A,k=1,v0=Theta,which="SA")

    # Compute 2-site Wave Function
    Theta_p=v[:,0].reshape(k[j]*d,k[j+2]*d)

    # Truncate the wave function
    A,lam,B=truncate(j,Theta_p,k,d=2)
    
    # Update A
    Ae[j]=A
    lamb[j+1]=lam

    vi[j+1]=np.divide(1.0, lam, out=np.zeros_like(lam), where=np.abs(lam)>=1E-14)

    Ae[j+1]=np.einsum("a,aib,b->aib",lam,B,vi[j+2])
    Be[j+1]=np.einsum("a,aib,b->aib",vi[j],A,lam)
    # Update L_env
    L_env[j+1]=np.einsum("abc,ajd,beji,cif->def",L_env[j],A,W,np.conj(A))
    L_env[j+1]=np.einsum("abc,ajd->jdbc",L_env[j],A)
    L_env[j+1]=np.einsum("jdbc,beji->deci",L_env[j+1],W)
    L_env[j+1]=np.einsum("deci,cif->def",L_env[j+1],np.conj(A))
  #left sweep
  for j in range(L-2,-1,-1):

    H=np.einsum("abc,bdij->adijc",L_env[j],W)
    H=np.einsum("adijc,dekl->aekl ijc",H,W)
    H=np.einsum("aekl ijc,feg->aikf cjlg",H,R_env[j+2]).reshape((k[j]*k[j+2]*d**2,k[j]*k[j+2]*d**2))

    A = LinearOperator((k[j]*k[j+2]*d**2,k[j]*k[j+2]*d**2), matvec=H_mult)
    Theta=np.einsum("a,aib,bjc->aijc",lamb[j],Be[j+1],Be[j+2]).reshape(k[j]*k[j+2]*d**2)
    
    w, v = eigsh(A,k=1,v0=Theta,which="SA")
    #w,v=LA.eigh(H)
    if LA.norm(H-H.T.conj())>0.000001:
      print("Not hermitian")
    Theta_p=v[:,0].reshape(k[j]*d,k[j+2]*d)
   
    A,lam,B=truncate(j,Theta_p,k,d=2)
   
    Be[j+2]=B
    lamb[j+1]=lam

    vi[j+1]=np.divide(1.0, lam, out=np.zeros_like(lam), where=np.abs(lam)>=1E-14)

    Be[j+1]=np.einsum("a,aib,b->aib",vi[j],A,lam)
    Ae[j+1]=np.einsum("a,aib,b->aib",lam,B,vi[j+2])
  
    # Update R_env
    R_env[j+1]=np.einsum("dja,ebji->debia",B,W)
    R_env[j+1]=np.einsum("debia,fic->debfca",R_env[j+1],np.conj(B))
    R_env[j+1]=np.einsum("debfca,abc->def",R_env[j+1],R_env[j+2])
    
  
J_zz = [[Jzz,i,i+1] for i in range(L-1)] 
h_x=[[hx,i] for i in range(L)]
static = [["zz",J_zz],["x",h_x]]
dynamic=[]
H_ZZX = hamiltonian(static,dynamic,basis=basis)

FsB=B_to_Fs(Be,L)
FsA=A_to_Fs(Ae,L)

exp_A = H_ZZX.expt_value(FsA).real
exp_B = H_ZZX.expt_value(FsB).real

print("Minimized energy - A",exp_A)
print("Minimized energy - B",exp_B)


Hermiticity check passed!
Symmetry checks passed!
Minimized energy - A -9.723207883119006
Minimized energy - B -9.723207883119013


In [3]:
E,V=H_ZZX.eigh()
#E_min=E[0]
print("Smallest eigenvalue ",E[0])
eha=H_ZZX.expt_value(V[:,0])
print(eha)
#print(eha.real,"exp_value")

Smallest eigenvalue  -9.72320788311896
(-9.723207883118972+0j)
